In [1]:
from Exercise import Exercise, MarkdownBlock
from config import URL, TOKEN
import json

import numpy as np
import sympy as sp
import matplotlib.pyplot as plt
import pandas as pd
plt.rcParams.update({'font.size': 20})

from sklearn.datasets import load_digits
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


Exercise.URL = URL
Exercise.TOKEN = TOKEN

In [121]:
a = np.arange(4)
b = -1 * np.arange(1, 5)

a = sp.Matrix([Rational(1/2), Rational(3/8), Rational(2, 5), Rational(1/1)])
b = sp.Matrix(b)

u = lambda x: sp.UnevaluatedExpr(x) 

u(u(a) + u(b))

from sympy import Add, Mul
from sympy import * 

def explain_add(a, b):
    assert(np.shape(a) == np.shape(b))
    rows, columns = np.shape(a)
    result = np.empty(shape=(rows, columns), dtype="object")
    for i in range(rows):
        for j in range(columns): 
              # Arguably, this is a hack
              result[i,j] = Symbol(f"({latex(u(a[i,j]))} + {latex(u(b[i,j]))})")
    return u(sp.Matrix(result))

def def_add(rows, columns):
    result = np.empty(shape=(rows, columns), dtype="object")
    a = np.empty(shape=(rows, columns), dtype="object")
    b = np.empty(shape=(rows, columns), dtype="object")
    for i in range(rows):
        for j in range(columns): 
              a[i, j] = Symbol(f"a_{{{i}, {j}}}")
              result[i,j] = Symbol(f"a_{{i, j}} + b_{{i, j}}")
    return u(sp.Matrix(a))

display(def_add(3,3))
    

# display(a)
# display(b)

# display(explain_add(a, b))

s = "What is $@a + @b?$"

params = {}
params["a"] = a
params["b"] = b
e = Exercise(MarkdownBlock(s, params))

e.display()

s1 = "Yes, $@a + @b = @c = @d$!"

params["c"] = explain_add(a,b)
params["d"] = a + b
e.add_answer(1, True, MarkdownBlock(s1, params))
e.evaluate_answer(params["d"])
e.write()
e.publish()

Matrix([
[a_{0, 0}, a_{0, 1}, a_{0, 2}],
[a_{1, 0}, a_{1, 1}, a_{1, 2}],
[a_{2, 0}, a_{2, 1}, a_{2, 2}]])

Published succesfully, preview at: https://www.mscthesis.nl/preview?id=5b664f34-9c27-4f99-a007-76ff55e68920
